# Loading Data

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

In [28]:
def read_data(trainfile='MNIST_train.csv', validationfile='MNIST_validation.csv', testfile='MNIST_test.csv'):
    
    dftrain = pd.read_csv(trainfile)
    dfval = pd.read_csv(validationfile)
    dftest = pd.read_csv(testfile)

    featurecols = list(dftrain.columns)
    featurecols.remove('label')
    featurecols.remove('even')

    targetcol1 = 'label'
    targetcol2 = 'even'

    Xtrain = dftrain[featurecols]
    ytrain = dftrain[targetcol1]
    ytrain2 = dftrain[targetcol2]
    
    Xval = dfval[featurecols]
    yval = dfval[targetcol1]
    yval2 = dfval[targetcol2]

    Xtest = dftest[featurecols]
    ytest = dftest[targetcol1]

    return (Xtrain, Xval, Xtest, ytrain, yval, ytest, ytrain2, yval2)

In [29]:
Xtrain, Xval, Xtest, ytrain, yval, ytest, ytrain2, yval2 = read_data()

# PCA

In [4]:
class PCA:
    """PCA class that can be fitted once and reused"""
    def __init__(self, variance_threshold=0.95, n_components=None):
        self.variance_threshold = variance_threshold
        self.n_components = n_components
        self.components = None
        self.mean = None
        self.explained_variance_ratio = None
        
    def fit(self, X):
        """Fit PCA on training data only"""
        # Normalize and center data
        X_normalized = np.array(X, dtype=float) / 255.0
        self.mean = np.mean(X_normalized, axis=0)
        X_centered = X_normalized - self.mean
        
        # Compute covariance matrix and eigen decomposition
        evd_matrix = (X_centered.T @ X_centered) / (X_centered.shape[0] - 1)
        eigenvalues, eigenvectors = np.linalg.eigh(evd_matrix)
        
        # Sort by descending eigenvalues
        sorted_idx = np.argsort(eigenvalues)[::-1]
        eigenvalues_sorted = eigenvalues[sorted_idx]
        eigenvectors_sorted = eigenvectors[:, sorted_idx]
        
        # Determine components for target variance
        self.explained_variance_ratio = eigenvalues_sorted / np.sum(eigenvalues_sorted)
        cumulative_variance = np.cumsum(self.explained_variance_ratio)

        if self.n_components is None:
            self.n_components = np.argmax(cumulative_variance >= self.variance_threshold) + 1
        
        # Store components
        self.components = eigenvectors_sorted[:, :self.n_components]
        
        print(f"PCA fitted with {self.n_components} components, explaining {cumulative_variance[self.n_components-1]:.4f} variance")
        return self
        
    def transform(self, X):
        """Transform data using fitted PCA"""
        if self.components is None:
            raise ValueError("PCA must be fitted before transforming")
            
        # Apply same normalization and centering
        X_normalized = np.array(X, dtype=float) / 255.0
        X_centered = X_normalized - self.mean
        
        # Project to PCA space
        return X_centered @ self.components
    
    def fit_transform(self, X):
        """Fit and transform in one step"""
        return self.fit(X).transform(X)

In [5]:
# Transform data to low dimensions
pca = PCA()
Xtrain_pca = pca.fit_transform(Xtrain)
Xval_pca = pca.transform(Xval)

# Ready for k-NN classification
print(f"Reduced from 784 to {Xtrain_pca.shape[1]} dimensions")

PCA fitted with 152 components, explaining 0.9501 variance
Reduced from 784 to 152 dimensions


# KNN

In [6]:
class KNN:
    def __init__(self, k=5, distance_metric='euclidean', weights='uniform'):
        self.k = k
        self.distance_metric = distance_metric
        self.weights = weights
        
    def fit(self, X, y):
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y)
        self.classes = np.unique(y)
        return self

    def _compute_all_distances(self, X):
        """
        Compute distances between X (n_test × d) and train (n_train × d)
        Fully vectorized
        """
        X = np.asarray(X)

        if self.distance_metric == 'euclidean':
            # (x - y)^2 = x^2 + y^2 - 2xy
            X2 = np.sum(X**2, axis=1).reshape(-1, 1)
            T2 = np.sum(self.X_train**2, axis=1).reshape(1, -1)
            dist = np.sqrt(np.maximum(X2 + T2 - 2 * X @ self.X_train.T, 0))

        elif self.distance_metric == 'manhattan':
            # |x - y|
            # Use broadcasting
            dist = np.sum(np.abs(X[:, None, :] - self.X_train[None, :, :]), axis=2)

        elif self.distance_metric == 'cosine':
            # 1 - cosine similarity
            X_norm = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-15)
            T_norm = self.X_train / (np.linalg.norm(self.X_train, axis=1, keepdims=True) + 1e-15)
            dist = 1 - X_norm @ T_norm.T

        else:
            raise ValueError("Unsupported metric")

        return dist

    def predict(self, X):
        X = np.asarray(X)

        # vectorized distance computation
        dist = self._compute_all_distances(X)

        # get k nearest neighbors
        idx = np.argpartition(dist, self.k, axis=1)[:, :self.k]
        neighbors = self.y_train[idx]

        if self.weights == 'uniform':
            # majority vote
            return np.array([Counter(row).most_common(1)[0][0] for row in neighbors])

        else:  # distance-weighted
            kdist = np.take_along_axis(dist, idx, axis=1)
            weights = 1.0 / (kdist + 1e-15)
            preds = []
            for lbls, w in zip(neighbors, weights):
                vote = {}
                for label, weight in zip(lbls, w):
                    vote[label] = vote.get(label, 0) + weight
                preds.append(max(vote.items(), key=lambda a: a[1])[0])
            return np.array(preds)

    def predict_proba(self, X):
        X = np.asarray(X)
        dist = self._compute_all_distances(X)

        idx = np.argpartition(dist, self.k, axis=1)[:, :self.k]
        neighbors = self.y_train[idx]

        probs = np.zeros((len(X), len(self.classes)))

        for i, row in enumerate(neighbors):
            for j, cls in enumerate(self.classes):
                probs[i, j] = np.sum(row == cls) / self.k

        return probs

In [7]:
# model = KNN(k=1)
# model.fit(Xtrain_pca, ytrain)
# ypred = model.predict(Xval_pca)
# print(accuracy_score(yval, ypred))
# print(confusion_matrix(yval, ypred))

# XGBoost Class

In [8]:
class TreeNode:
    """Simple tree node structure"""
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

In [9]:
class XGBoostClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3,
                 lambda_l2=1, gamma=0, min_child_weight=1, threshold=0.5):
        
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.lambda_l2 = lambda_l2
        self.gamma = gamma
        self.min_child_weight = min_child_weight
        self.threshold = threshold
        
        self.trees = []
        self.initial_prediction = None


    # ================================
    # MATH
    # ================================
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -12, 12)))

    def log_loss_derivatives(self, y_true, y_pred):
        p = self.sigmoid(y_pred)
        grad = p - y_true
        hess = p * (1 - p)
        return grad, hess

    def calc_gain(self, G, H):
        return (G * G) / (H + self.lambda_l2)

    def compute_gain(self, G, H, G_left, H_left, G_right, H_right):
        gain = 0.5 * (self.calc_gain(G_left, H_left) +
                      self.calc_gain(G_right, H_right) -
                      self.calc_gain(G, H)) - self.gamma
        return gain


    # ================================
    # BEST SPLIT FOR ONE FEATURE
    # ================================
    def best_split_feature(self, X, grad, hess, feature, sorted_idx):
        fv = X[sorted_idx, feature]

        # cumulative sums
        G_cum = np.cumsum(grad[sorted_idx])
        H_cum = np.cumsum(hess[sorted_idx])

        G_total = G_cum[-1]
        H_total = H_cum[-1]

        best_gain = 0
        best_thr = None

        for i in range(len(sorted_idx) - 1):

            # skip identical values
            if fv[i] == fv[i + 1]:
                continue

            G_left = G_cum[i]
            H_left = H_cum[i]

            G_right = G_total - G_left
            H_right = H_total - H_left

            if H_left < self.min_child_weight or H_right < self.min_child_weight:
                continue

            gain = self.compute_gain(G_total, H_total, G_left, H_left, G_right, H_right)

            if gain > best_gain:
                best_gain = gain
                best_thr = (fv[i] + fv[i + 1]) / 2

        return best_gain, best_thr


    # ================================
    # BEST SPLIT ACROSS ALL FEATURES
    # ================================
    def best_split(self, X, grad, hess):
        n_samples, n_features = X.shape

        # Feature subsampling (sqrt rule)
        feat_count = int(np.sqrt(n_features))
        features = np.random.choice(n_features, feat_count, replace=False)

        best_gain = 0
        best_feat = None
        best_thr = None

        # Compute sorted indices LOCALLY for this node
        local_sorted = {f: np.argsort(X[:, f]) for f in features}

        for f in features:
            gain, thr = self.best_split_feature(X, grad, hess, f, local_sorted[f])

            if thr is not None and gain > best_gain:
                best_gain = gain
                best_feat = f
                best_thr = thr

        return best_feat, best_thr


    # ================================
    # TREE BUILDING (RECURSIVE)
    # ================================
    def build_tree(self, X, grad, hess, depth):

        if (depth >= self.max_depth or
            len(X) < 2 or
            hess.sum() < self.min_child_weight):

            leaf_val = -grad.sum() / (hess.sum() + self.lambda_l2)
            return TreeNode(value=leaf_val)

        # best split
        feat, thr = self.best_split(X, grad, hess)

        if feat is None:
            leaf_val = -grad.sum() / (hess.sum() + self.lambda_l2)
            return TreeNode(value=leaf_val)

        left_mask = (X[:, feat] <= thr)
        right_mask = ~left_mask

        left = self.build_tree(X[left_mask], grad[left_mask], hess[left_mask], depth + 1)
        right = self.build_tree(X[right_mask], grad[right_mask], hess[right_mask], depth + 1)

        return TreeNode(feature_index=feat, threshold=thr, left=left, right=right)


    # ================================
    # TREE PREDICTION
    # ================================
    def predict_tree(self, X, node):
        out = np.zeros(len(X))

        for i, x in enumerate(X):
            cur = node
            while cur.value is None:
                if x[cur.feature_index] <= cur.threshold:
                    cur = cur.left
                else:
                    cur = cur.right
            out[i] = cur.value

        return out


    # ================================
    # FIT
    # ================================
    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        # initial prediction = log odds
        p = np.mean(y)
        self.initial_prediction = np.log(p / (1 - p + 1e-12))

        pred = np.full(len(y), self.initial_prediction)

        self.trees = []

        for _ in range(self.n_estimators):

            grad, hess = self.log_loss_derivatives(y, pred)

            tree = self.build_tree(X, grad, hess, depth=0)
            self.trees.append(tree)

            # vectorized update
            pred += self.learning_rate * self.predict_tree(X, tree)


    # ================================
    # PREDICT
    # ================================
    def predict_proba(self, X):
        X = np.asarray(X)

        pred = np.full(len(X), self.initial_prediction)

        for tree in self.trees:
            pred += self.learning_rate * self.predict_tree(X, tree)

        p = self.sigmoid(pred)
        return np.column_stack([1 - p, p])

    def predict(self, X):
        p = self.predict_proba(X)[:, 1]
        return (p >= self.threshold).astype(int)


# SVM

In [10]:
class SVM:
    def __init__(self, learning_rate=0.001, lambda_p =0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_p = lambda_p
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        # Convert labels to -1/+1
        y_ = np.where(y <= 0, -1, 1)
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # only regularization penalty
                    self.w -= self.lr * (self.lambda_p * self.w)
                else:
                    # only regularization penalty
                    self.w -= self.lr * (self.lambda_p * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

# Final:

In [11]:
def run_hybrid_model_ovo_knn(Xtrain, ytrain, Xval, yval, ytrain_oe, yval_oe):

    print("\n================= PCA (GLOBAL) =================")
    pca_global = PCA()
    Xtrain_pca = pca_global.fit_transform(Xtrain)
    Xval_pca   = pca_global.transform(Xval)

    print("\n================= TRAIN KNN (GLOBAL) =================")
    knn_global = KNN(k=1)
    knn_global.fit(Xtrain_pca, ytrain)
    ypred_knn = knn_global.predict(Xval_pca)
    print("Base KNN accuracy:", accuracy_score(yval, ypred_knn))

    print("\n================= TRAIN XGB EVEN/ODD =================")
    model_xgb_even = XGBoostClassifier(
        n_estimators=105,
        learning_rate=0.3,
        max_depth=9,
        lambda_l2=0.1,
        gamma=0,
        min_child_weight=1,
        threshold=0.5
    )
    model_xgb_even.fit(Xtrain, ytrain_oe)
    ypred_xgb_even = model_xgb_even.predict(Xval)
    print("XGB Even/Odd accuracy:", accuracy_score(yval_oe, ypred_xgb_even))

    print("\n================= FIND PARITY MISTAKES =================")
    mistakes = []
    for i in range(len(yval)):
        if ypred_xgb_even[i] != (yval[i] % 2):
            mistakes.append(i)

    print(f"Total parity-mismatch candidates: {len(mistakes)}")


    # ==================================================================
    # ONE-VS-ONE KNN MODELS: 45 KNN MODELS + 45 PCA MODELS
    # ==================================================================
    print("\n================= TRAINING ONE-VS-ONE KNN MODELS =================")

    ovo_knn = {}     # (digitA, digitB) -> KNN model
    ovo_pca = {}     # (digitA, digitB) -> PCA

    digits = list(range(10))

    for i in digits:
        for j in digits:
            if i < j:
                mask = (ytrain == i) | (ytrain == j)
                X_pair = Xtrain[mask]
                y_pair = ytrain[mask]

                pca_pair = PCA()
                X_pair_pca = pca_pair.fit_transform(X_pair)

                knn_pair = KNN(k=1)
                knn_pair.fit(X_pair_pca, y_pair)

                ovo_knn[(i, j)] = knn_pair
                ovo_pca[(i, j)] = pca_pair

    print("All 45 one-vs-one KNN models trained.")

    # ==================================================================
    # HYBRID CORRECTION USING OVO-KNN FOR MISMATCHES
    # ==================================================================
    print("\n================= APPLY HYBRID OVO-KNN CORRECTION =================")

    y_final = ypred_knn.copy()

    for idx in mistakes:

        pred = ypred_knn[idx]
        xgb_par = ypred_xgb_even[idx]   # 0=even, 1=odd
        # true parity = yval[idx] % 2

        # Step 1: only correct digits with parity conflict
        # Step 2: pick opponent class based on XGB parity
        if xgb_par == 0:
            # XGB says EVEN → candidate even digits {0,2,4,6,8}
            candidates = [0,2,4,6,8]
        else:
            # XGB says ODD → candidate odd digits {1,3,5,7,9}
            candidates = [1,3,5,7,9]

        # Step 3: Run pairwise comparisons: pred vs each candidate
        votes = {}

        for d in candidates:
            if d == pred:
                continue

            a, b = min(pred, d), max(pred, d)

            # Transform with this pair's PCA
            xval_p = ovo_pca[(a, b)].transform(Xval.iloc[idx:idx+1])
            vote = ovo_knn[(a, b)].predict(xval_p)[0]

            votes[d] = vote

        # Step 4: pick the most common voted label
        if len(votes.values()) > 0:
            best = max(votes.values(), key=list(votes.values()).count)
            y_final[idx] = best


    # ==================================================================
    # RESULTS
    # ==================================================================
    print("\n================= FINAL RESULTS =================")
    acc = accuracy_score(yval, y_final)
    print("Final hybrid accuracy:", acc)
    print(confusion_matrix(yval, y_final))

    return y_final, ypred_knn, ypred_xgb_even

In [12]:
# run_hybrid_model_ovo_knn(Xtrain, ytrain, Xval, yval, ytrain_oe, yval_oe)

In [13]:
def train_ovo_knn_models(Xtrain, ytrain):
    """
    Trains 45 one-vs-one (i<j) PCA + KNN(k=1) models.
    Returns:
        ovo_knn[(i,j)] = trained KNN
        ovo_pca[(i,j)] = trained PCA
    """
    print("\n================= TRAINING 45 OVO KNN MODELS =================")

    ovo_knn = {}
    ovo_pca = {}

    digits = list(range(10))

    for i in digits:
        for j in digits:
            if i < j:

                # Select samples belonging to digit i or j
                mask = (ytrain == i) | (ytrain == j)
                X_pair = Xtrain[mask]
                y_pair = ytrain[mask]

                # PCA for this pair
                pca_pair = PCA(n_components=30)
                X_pair_pca = pca_pair.fit_transform(X_pair)

                # KNN for this pair
                knn_pair = KNN(k=1)
                knn_pair.fit(X_pair_pca, y_pair)

                ovo_knn[(i, j)] = knn_pair
                ovo_pca[(i, j)] = pca_pair

                print(f"✓ OVO model trained for pair ({i}, {j}) | Samples: {len(X_pair)}")

    print("\nAll 45 OVO KNN models trained successfully.")
    return ovo_knn, ovo_pca


In [14]:
def ovo_knn_predict(ovo_knn, ovo_pca, X):
    """
    Predict using all 45 OVO KNN models with majority voting.
    """
    n = X.shape[0]
    y_pred = np.zeros(n, dtype=int)

    print("\n================= OVO-KNN PREDICTION =================")

    for idx in range(n):
        x = X.iloc[idx:idx+1]

        votes = []

        for (i, j), knn_model in ovo_knn.items():
            pca_model = ovo_pca[(i, j)]

            # project x into PCA space for this pair
            x_pca = pca_model.transform(x)

            # predict i or j
            vote = knn_model.predict(x_pca)[0]
            votes.append(vote)

        # majority vote
        y_pred[idx] = max(set(votes), key=votes.count)

        if idx % 200 == 0:
            print(f"Predicted {idx}/{n}")

    return y_pred


In [15]:
def run_ovo_knn_pipeline(train_csv='MNIST_train.csv', val_csv='MNIST_validation.csv'):

    # Load data
    Xtrain, ytrain, Xval, yval, ytrain_oe, yval_oe = read_data(train_csv, val_csv)

    # Train 45 OVO models
    ovo_knn, ovo_pca = train_ovo_knn_models(Xtrain, ytrain)

    # Predict on validation set
    y_pred = ovo_knn_predict(ovo_knn, ovo_pca, Xval)

    # Show results
    acc = accuracy_score(yval, y_pred)
    cm  = confusion_matrix(yval, y_pred)

    print("\n================= FINAL OVO-KNN RESULTS =================")
    print("Accuracy:", acc)
    print(cm)

    return y_pred, ovo_knn, ovo_pca


In [16]:
y_pred, ovo_knn, ovo_pca = run_ovo_knn_pipeline()


================= TRAINING 45 OVO KNN MODELS =================
PCA fitted with 30 components, explaining 0.8550 variance
✓ OVO model trained for pair (0, 1) | Samples: 2111
PCA fitted with 30 components, explaining 0.7797 variance
✓ OVO model trained for pair (0, 2) | Samples: 1980
PCA fitted with 30 components, explaining 0.7887 variance
✓ OVO model trained for pair (0, 3) | Samples: 2009
PCA fitted with 30 components, explaining 0.7946 variance
✓ OVO model trained for pair (0, 4) | Samples: 1961
PCA fitted with 30 components, explaining 0.7819 variance
✓ OVO model trained for pair (0, 5) | Samples: 1891
PCA fitted with 30 components, explaining 0.8102 variance
✓ OVO model trained for pair (0, 6) | Samples: 1974
PCA fitted with 30 components, explaining 0.8044 variance
✓ OVO model trained for pair (0, 7) | Samples: 2031
PCA fitted with 30 components, explaining 0.7756 variance
✓ OVO model trained for pair (0, 8) | Samples: 1962
PCA fitted with 30 components, explaining 0.8058 varianc

In [17]:
def train_ovr_knn_models(Xtrain, ytrain):
    """
    Train 10 One-Vs-Rest PCA + KNN(k=1) models.
    
    Returns:
        ovr_knn[d] = KNN for digit d vs rest
        ovr_pca[d] = PCA for digit d vs rest
    """

    print("\n================= TRAINING 10 OVR KNN MODELS =================")

    ovr_knn = {}
    ovr_pca = {}

    digits = list(range(10))

    for d in digits:

        # Mask: digit = d (positive) or not d (negative)
        mask = (ytrain == d)
        X_pos = Xtrain[mask]
        X_neg = Xtrain[~mask]

        # Build OVR dataset: keep all positives + a sampled set of negatives
        # (optional: downsample negatives to speed up)
        # X_comb = np.vstack([X_pos, X_neg])
        # y_comb = np.concatenate([np.ones(len(X_pos)), np.zeros(len(X_neg))])

        X_comb = Xtrain  # FULL dataset (best accuracy)
        y_comb = (ytrain == d).astype(int)

        # PCA for this class
        pca_d = PCA(n_components=30)
        X_pca = pca_d.fit_transform(X_comb)

        # Train KNN
        knn_d = KNN(k=1)
        knn_d.fit(X_pca, y_comb)

        ovr_knn[d] = knn_d
        ovr_pca[d] = pca_d

        print(f"✓ OVR model trained for digit {d} | Positives={len(X_pos)}")

    print("\nAll 10 OVR KNN models trained successfully.")
    return ovr_knn, ovr_pca


In [30]:
def ovr_knn_predict(ovr_knn, ovr_pca, X):
    """
    Predict digits using 10 One-Vs-Rest KNN models.
    Works for VAL or TEST.
    """
    n = X.shape[0]
    y_pred = np.zeros(n, dtype=int)

    print("\n================= OVR-KNN PREDICTION =================")

    for idx in range(n):

        x = X.iloc[idx:idx+1]
        scores = {}

        for d in range(10):
            pca_d = ovr_pca[d]
            knn_d = ovr_knn[d]

            # PCA transform
            x_pca = pca_d.transform(x)

            # 1 = positive class → used as confidence
            pred = knn_d.predict(x_pca)[0]
            scores[d] = pred

        # pick max-score digit
        y_pred[idx] = max(scores, key=scores.get)

        if idx % 200 == 0:
            print(f"Predicted {idx}/{n}")

    return y_pred

In [53]:
def run_ovr_knn_pipeline(train_csv='MNIST_train.csv',
                         val_csv='MNIST_validation.csv',
                         test_csv='MNIST_test.csv'):

    # --- Load data ---
    (Xtrain, Xval, Xtest,
     ytrain, yval, ytest,
     ytrain2, yval2) = read_data(train_csv, val_csv, test_csv)

    # --- Train OVR ---
    ovr_knn, ovr_pca = train_ovr_knn_models(Xtrain, ytrain)

    # --- Predict VAL ---
    y_pred_val = ovr_knn_predict(ovr_knn, ovr_pca, Xval)

    print("\n================= VALIDATION RESULTS =================")
    print("Accuracy:", accuracy_score(yval, y_pred_val))
    print(confusion_matrix(yval, y_pred_val))

    # --- Predict TEST ---
    y_pred_test = ovr_knn_predict(ovr_knn, ovr_pca, Xtest)

    print("\n================= TEST RESULTS =================")
    print("Accuracy:", accuracy_score(ytest, y_pred_test))
    print(confusion_matrix(ytest, y_pred_test))

    return y_pred_val, y_pred_test, ovr_knn, ovr_pca


In [ ]:
run_ovr_knn_pipeline()


================= TRAINING 10 OVR KNN MODELS =================
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 0 | Positives=987
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 1 | Positives=1124
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 2 | Positives=993
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 3 | Positives=1022
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 4 | Positives=974
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 5 | Positives=904
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 6 | Positives=987
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for digit 7 | Positives=1044
PCA fitted with 30 components, explaining 0.7306 variance
✓ OVR model trained for dig